In [1]:
# Import libralis
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.display import clear_output

In [2]:
def get_url(city_name, num_post):
    return f'https://{city_name.lower()}.craigslist.org/search/apa?s={num_post}'

# Chilliwack rental housing

response = get(get_url(city_name = 'Abbotsford', num_post = 120))


soup = BeautifulSoup(response.text, 'html.parser')

posts = soup.find_all('li', class_='result-row')
print(type(posts)) # to double check that I got resultSet
print(len(posts)) # to double check I got 120 elements

<class 'bs4.element.ResultSet'>
120


In [3]:
# Show first post to look into it
post_one = posts[0]
post_one

<li class="result-row" data-pid="7561022887">
<a class="result-image gallery" data-ids="3:00H0H_61Mv94VrUO6z_0CI0lM,3:00303_4bYXyHLOre3z_0CI0lM,3:00r0r_KQ7lji3hEwz_0CI0lM,3:00t0t_izgcbn38fZKz_0lM0CI,3:00b0b_ls5eHGzUIgCz_0lM0CI,3:00r0r_7pnqoRnBhh1z_0CI0lM,3:01515_5xQIRdlgO0Zz_0lM0CI,3:00h0h_fSvvUdbNpE8z_0CI0lM,3:00u0u_hSk1ulaxOqqz_0lM0CI,3:00000_5A1fFNjC1NLz_0lM0CI,3:00000_3PT1l51nTJzz_0CI0lM,3:00Y0Y_cfvHKXVZbAsz_0CI0lM,3:00V0V_e1FmauSs65hz_0CI0lM,3:00g0g_1R6JPeUj4bQz_0CI0lM" href="https://abbotsford.craigslist.org/apa/d/abbotsford-southeast-luxury-loft-bed-den/7561022887.html">
<span class="result-price">$1,850</span>
</a>
<div class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">favorite this post</span>
</span>
<time class="result-date" datetime="2022-12-06 16:44" title="Tue 06 Dec 04:44:17 PM">Dec  6</time>
<h3 class="result-heading">
<a class="result-title hdrlnk" data-id="7561022887" href="https://abbotsford.craigslist.org/apa/d/abbotsf

In [4]:
# City posted
post_city = post_one.find('span', class_='result-hood').text.strip('( )')
post_city

'Abbotsford'

In [5]:
# Number of bedroom
num_bed = int(post_one.find('span', class_='housing').text.split()[0].replace('br', ''))
num_bed, type(num_bed)

(1, int)

In [6]:
# SQFT
sqft = int(post_one.find('span', class_='housing').text.split()[2].replace('ft2', ''))
sqft, type(sqft)

(900, int)

In [7]:
# rental price
price = int(post_one.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
price, type(price)

(1850, int)

In [8]:
# Post ad title
post_ad = post_one.find('a', class_='result-title hdrlnk').text
post_ad

'Luxury Loft 1 bed + Den'

In [9]:
# post datetime
datetime = datetime.strptime(post_one.find('time', class_='result-date')['datetime'], '%Y-%m-%d %H:%M')
datetime, type(datetime)

(datetime.datetime(2022, 12, 6, 16, 44), datetime.datetime)

In [10]:
search_legend = soup.find('div', class_='search-legend')
search_legend

<div class="search-legend">
<div class="search-view js-only">
<div aria-expanded="false" aria-label="view options" class="dropdown dropdown-icons dropdown-arrows dropdown-view" role="toolbar">
<ul class="dropdown-list">
<li class="dropdown-item mode sel">
<button data-selection="list" id="listview" title="show results in a list">
<span class="view-list icon"></span>list
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="pic" id="picview" title="show results in a list with thumbnail pictures">
<span class="view-thumb icon"></span>thumb
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="grid" id="gridview" title="show results side-by-side with larger pictures">
<span class="view-gallery icon"></span>gallery
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="map" id="mapview" 

In [11]:
# Find out the beggining of the post number 
num_post_start = search_legend.find('span', class_='rangeFrom').text
num_post_start

'121'

In [12]:
# Find out total number of the search result
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
results_total

1532